# Predicción de la producción de activos industriales utilizando el algoritmo de detección de anomalías de Azure

### 1. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
import math

from azure.cognitiveservices.anomalydetector import AnomalyDetectorClient
from azure.cognitiveservices.anomalydetector.models import Request, Point, Granularity, \
    APIErrorException
from msrest.authentication import CognitiveServicesCredentials
import os

from sklearn.metrics import mean_squared_error

### 2. Conexión con Azure y autentificación del cliente

In [3]:
# URLs para la detección de anomalías
batch_detection_url = "/anomalydetector/v1.0/timeseries/entire/detect"

# Se debe crear la variable de entorno y el endpoint correspondiente según la sinstrucciones de Azure
endpoint = os.environ["ANOMALY_DETECTOR_ENDPOINT"]
subscription_key = os.environ["ANOMALY_DETECTOR_KEY"]

#Autentificación del cliente
client=AnomalyDetectorClient(endpoint, CognitiveServicesCredentials(subscription_key))

### 3. Lectura de datos y preparación

In [8]:
Y_train=pd.read_csv(('Datos/train_y.csv'))
Y_train

,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,3850220.0,9019860.320
1,2,3793240.0,8568867.304
2,3,3870570.0,8549258.912
3,4,3801380.0,8451216.952
4,5,3752540.0,8323762.404
...,...,...,...
99,100,3984530.0,9372811.376
100,101,3976390.0,9588503.688
101,102,3903130.0,9598307.884
102,103,3927550.0,9745370.824


In [9]:

data=pd.read_csv('Output/X_train_formateado.csv')  #Los datos originados fueron preprocesados en el archivo 1. Detección
                                                   #de outiliers Holt-Winter
data

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5
...,...,...,...,...,...,...,...,...,...,...,...,...
60419,83,3,104,3,1.0,0.0,48.0,8.0,4,104_3,2019-12-25,724
60420,83,3,104,4,0.0,0.0,37.0,1.0,4,104_4,2019-12-26,725
60421,83,3,104,5,1.0,0.0,48.0,9.0,4,104_5,2019-12-27,726
60422,83,3,104,6,2.0,0.0,53.0,11.0,4,104_6,2019-12-28,727


In [10]:
data['borrar']='T00:00:00Z'
data['date']=data['date']+data['borrar']
data['date']=pd.to_datetime(data['date'])
data=data.drop('borrar', axis=1)
data

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01 00:00:00+00:00,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02 00:00:00+00:00,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03 00:00:00+00:00,3
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04 00:00:00+00:00,4
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05 00:00:00+00:00,5
...,...,...,...,...,...,...,...,...,...,...,...,...
60419,83,3,104,3,1.0,0.0,48.0,8.0,4,104_3,2019-12-25 00:00:00+00:00,724
60420,83,3,104,4,0.0,0.0,37.0,1.0,4,104_4,2019-12-26 00:00:00+00:00,725
60421,83,3,104,5,1.0,0.0,48.0,9.0,4,104_5,2019-12-27 00:00:00+00:00,726
60422,83,3,104,6,2.0,0.0,53.0,11.0,4,104_6,2019-12-28 00:00:00+00:00,727


In [11]:
#Leer datos de capacidad nominal por asset por semana

capacity=pd.read_csv('Datos/assets.csv', header=0, names=['asset_id', 'nominalc_week'],
                     dtype={'asset_id':int, 'nominalc_week':float} )

#agregar columna de capacidad por día
capacity['nominalc_day']=capacity['nominalc_week']/7     
capacity.head()

,asset_id,nominalc_week,nominalc_day
0,1,132000.0,18857.142857
1,2,93000.0,13285.714286
2,3,238600.0,34085.714286
3,4,206000.0,29428.571429
4,5,310000.0,44285.714286


In [12]:
data=data.merge(capacity[['asset_id','nominalc_day']])
data.head()


,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day,nominalc_day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01 00:00:00+00:00,1,18857.142857
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02 00:00:00+00:00,2,18857.142857
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03 00:00:00+00:00,3,18857.142857
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04 00:00:00+00:00,4,18857.142857
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05 00:00:00+00:00,5,18857.142857


### 3. Definición de funciones

In [13]:
 
def ClasificacionDeObservaciones(df):  
    '''FUNCIÓN PARA CLASIFICAR ANOMALÍAS MEDIANTE TRANSFORMADA DE WAVELET PARA CADA UNA 
    DE LAS SERIES CORREPONDIENTES A LAS MEDICIONES HECHAS EN CADA ASSET
    df: pandas data frame
    wavelet_family:  'haar', 'db2', o cualquier otra
    '''

    if df.total_t.mean() == 2:
        mediciones=[3,4]
    else:
        mediciones=[1,2,3,4]
    for x in mediciones:  #para cada una de las mediciones
        t='t'+str(x)
        if df[t].unique().sum()!=0:
            series=[]
            for index, row in df.iterrows():
                series.append(Point(timestamp=row['date'].isoformat(),value=row[t]))
            request= Request(series=series, granularity=Granularity.daily, period=7, sensitivity=20)
            try:
                response = client.entire_detect(request)
            except Exception as e:
                if isinstance(e, APIErrorException):
                    print('Error code: {}'.format(e.error.code),
                          'Error message: {}'.format(e.error.message))
                else:
                    print(e)
            respuesta=response.as_dict()
            #Crear df anomlias con resultados en el diccionario respuesta
            for key, value in respuesta.items():
                anomalias=pd.DataFrame(respuesta)
            #Agregar columna a asset
            df['outlier_'+t]=anomalias['is_positive_anomaly'].astype(int)
        continue
    return df       

In [14]:
def resultados(dict_df):
    '''FUNCION PARA CALCULAR EL ERROR ASIGNANDO EL VALOR DE PRODUCCION NOMINAL DIARIO A LOS DÍAS EN QUE NO HUBO 
    NINGUNA ANOMALÍAS Y 0 EN CASO CONTRARIO 
    
    dict_df: diccionario resultante con la detección de anomalías con un dataframe por cada asset

    '''
    
    X=pd.concat(dict_df.values())
    X=X.reset_index(drop=True)
    
    X=X[['asset_id', 'group_id', 'week', 'weekday','total_t','nominalc_day', 'outlier_t1','outlier_t2','outlier_t3','outlier_t4']]
    

    X['produccion']=np.where(
        ( ( X['outlier_t1']==1 ) | ( X['outlier_t2']==1 ) | (X['outlier_t3']==1)  | ( X['outlier_t4']==1 ) ),
        0, X['nominalc_day'] )
   
    #Agregamos los datos por grupo y semana para obtener Y_predicted

    ProduccionGrupoSemana=X.groupby(['group_id','week'])
    Y_predicted=ProduccionGrupoSemana.agg({'produccion':'sum'})
    Y_predicted=Y_predicted.reset_index()

    # Configuramos el formato  Y_predicted para tener el mismo que en Y_train
    Y_predicted=pd.pivot_table(Y_predicted,index=['week'], columns=['group_id'], values=['produccion'])
    Y_predicted=Y_predicted.reset_index()
    Y_predicted.columns.set_levels(['production_group_2', 'production_group_3', ''], level=1, inplace=True)
    Y_predicted.columns=list(map("".join, Y_predicted))
    Y_predicted.columns=Y_train.columns

    # Calculamos el error para cada guropo
    MSE2=mean_squared_error(Y_train.PRODUCTION_GROUP_2, Y_predicted.PRODUCTION_GROUP_2)
    MSE3=mean_squared_error(Y_train.PRODUCTION_GROUP_3, Y_predicted.PRODUCTION_GROUP_3)

    
    #Calculamos el error total
    Y_train_tot = Y_train.PRODUCTION_GROUP_2.to_list() + Y_train.PRODUCTION_GROUP_3.to_list()
    Y_predicted_tot = Y_predicted.PRODUCTION_GROUP_2.to_list() +  Y_predicted.PRODUCTION_GROUP_3.to_list() 
    ErrorTotal = mean_squared_error(Y_train_tot, Y_predicted_tot)
    

    return Y_predicted, ErrorTotal

### 4. Clasificar observaciones, identificando anomalías para cada uno de las series de tiempo de los assets

In [15]:
#Crear objeto GroupBy de Pandas para iterar despues en las tablas de cada asset
data=data.set_index('asset_id')
porAsset=data.groupby(data.index)

In [16]:
%%time 
# 1 min 41 seg aproximadamente

#Pasamos el subconjunto de datos de cada asset por la función de deteccion de anomalías 
#y guardamos cada uno de los dataframes resultantes con la clasificacione de las observaciones en el diccionario new_data
import time
start_time=time.time()
new_data={}
for ass in range(1,84):
    asset = porAsset.get_group(ass).reset_index()
    new_data[ass] = ClasificacionDeObservaciones(asset)
print("--- %s seconds ---"%(time.time()-start_time))

--- 91.46268129348755 seconds ---
Wall time: 1min 31s


### 5. Resultados

In [17]:
y_predicted, error= resultados(new_data)
print('error =', np.format_float_scientific(error))
y_predicted

error = 2.0547269315712854e+12


,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,3.577851e+06,7.410714e+06
1,2,3.444366e+06,6.366856e+06
2,3,3.717009e+06,5.904107e+06
3,4,3.395196e+06,6.046025e+06
4,5,3.752381e+06,5.958516e+06
...,...,...,...
99,100,3.135859e+06,7.265531e+06
100,101,3.039880e+06,7.447671e+06
101,102,3.311820e+06,7.743620e+06
102,103,3.757094e+06,7.912013e+06


## Referencias


1. aahill. Quickstart: Use the Anomaly Detector client library - Azure Cognitive Services [Internet]. [citado 29 de julio de 2020]. Disponible en: https://docs.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/quickstarts/client-libraries


2. Hou X, Zhang L. Saliency detection: A spectral residual approach. En: In IEEE Conference on Computer Vision and Pattern Recognition (CVPR07) IEEE Computer Society. 2007. p. 1–8. Disponible en: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.125.5641&rep=rep1&type=pdf


3. Ren H, Xu B, Wang Y, Yi C, Huang C, Kou X, et al. Time-Series Anomaly Detection Service at Microsoft. Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining [Internet]. 25 de julio de 2019 [citado 30 de julio de 2020];3009-17. Disponible en: http://arxiv.org/abs/1906.03821

